In [1]:
%pwd

'd:\\ObjectDetection(project-1)\\Research'

In [2]:
import os 

In [3]:
os.chdir('../')

In [4]:
%pwd

'd:\\ObjectDetection(project-1)'

In [5]:
import os 
import sys
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
import xml.etree.ElementTree as ET
from shutil import copy,move
from PIL import Image
from glob import glob
sys.path.append(os.path.abspath(os.path.join(os.path.dirname(os.getcwd()), '../')))
from src.constant import CONFIG_FILE_PATH,PARAMS_FILE_PATH
from src.utils.common import read_yaml,create_directories
from src.logger  import  logging
from src.exception import CustomException

C:\Users\HP\AppData\Local\Temp\ipykernel_16132\2354208834.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [6]:
xml_file_path = glob('artifacts/data_ingestion/Face_Musk_Detection/Datatset/annotations/*.xml')

data_label = dict(filename = [],width = [],height=[],xmin = [],xmax = [],ymin = [],ymax = [],name = [])

for file in xml_file_path:
    tree = ET.parse(file)
    root = tree.getroot()
    filename = root.find('filename').text
    size = root.find('size')
    width = float(size.find('width').text)
    height = float(size.find('height').text)
    object_all = root.findall('object')
    for obj in object_all:
        name = obj.find('name').text
        bndbox = obj.find('bndbox')
        xmin = float(bndbox.find('xmin').text)
        xmax = float(bndbox.find('xmax').text)
        ymin = float(bndbox.find('ymin').text)
        ymax = float(bndbox.find('ymax').text)
        data_label['filename'].append(filename)
        data_label['width'].append(width)
        data_label['height'].append(height)
        data_label['xmin'].append(xmin)
        data_label['xmax'].append(xmax)
        data_label['ymin'].append(ymin)
        data_label['ymax'].append(ymax)
        data_label['name'].append(name)
    

In [7]:
df = pd.DataFrame(data_label)
df.head()

,filename,width,height,xmin,xmax,ymin,ymax,name
0,maksssksksss0.png,512.0,366.0,79.0,109.0,105.0,142.0,without_mask
1,maksssksksss0.png,512.0,366.0,185.0,226.0,100.0,144.0,with_mask
2,maksssksksss0.png,512.0,366.0,325.0,360.0,90.0,141.0,without_mask
3,maksssksksss1.png,400.0,156.0,321.0,354.0,34.0,69.0,with_mask
4,maksssksksss1.png,400.0,156.0,224.0,261.0,38.0,73.0,with_mask


In [8]:
# center_x,cencter_y,width,height
df['center_x'] = (df['xmin'] + df['xmax'])/(2 * df['width'])
df['center_y'] = (df['ymin'] + df['ymax'])/(2 * df['height'])
df['W'] = (df['xmax'] - df['xmin'])/(df['width'])
df['H'] = (df['ymax'] - df['ymin'])/(df['height'])

In [9]:
df.head()

,filename,width,height,xmin,xmax,ymin,ymax,name,center_x,center_y,W,H
0,maksssksksss0.png,512.0,366.0,79.0,109.0,105.0,142.0,without_mask,0.183594,0.337432,0.058594,0.101093
1,maksssksksss0.png,512.0,366.0,185.0,226.0,100.0,144.0,with_mask,0.401367,0.333333,0.080078,0.120219
2,maksssksksss0.png,512.0,366.0,325.0,360.0,90.0,141.0,without_mask,0.668945,0.315574,0.068359,0.139344
3,maksssksksss1.png,400.0,156.0,321.0,354.0,34.0,69.0,with_mask,0.843750,0.330128,0.082500,0.224359
4,maksssksksss1.png,400.0,156.0,224.0,261.0,38.0,73.0,with_mask,0.606250,0.355769,0.092500,0.224359


In [10]:
df['name'].value_counts()

name
with_mask                3232
without_mask              717
mask_weared_incorrect     123
Name: count, dtype: int64

In [11]:
soucrce_code = {
    'with_mask':0,
    'without_mask':1,
    'mask_weared_incorrect':2
}
df['label'] = df['name'].apply(lambda x: soucrce_code[x])

In [12]:
df

,filename,width,height,xmin,xmax,ymin,ymax,name,center_x,center_y,W,H,label
0,maksssksksss0.png,512.0,366.0,79.0,109.0,105.0,142.0,without_mask,0.183594,0.337432,0.058594,0.101093,1
1,maksssksksss0.png,512.0,366.0,185.0,226.0,100.0,144.0,with_mask,0.401367,0.333333,0.080078,0.120219,0
2,maksssksksss0.png,512.0,366.0,325.0,360.0,90.0,141.0,without_mask,0.668945,0.315574,0.068359,0.139344,1
3,maksssksksss1.png,400.0,156.0,321.0,354.0,34.0,69.0,with_mask,0.843750,0.330128,0.082500,0.224359,0
4,maksssksksss1.png,400.0,156.0,224.0,261.0,38.0,73.0,with_mask,0.606250,0.355769,0.092500,0.224359,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4067,maksssksksss98.png,400.0,267.0,263.0,287.0,62.0,85.0,with_mask,0.687500,0.275281,0.060000,0.086142,0
4068,maksssksksss98.png,400.0,267.0,344.0,377.0,80.0,106.0,with_mask,0.901250,0.348315,0.082500,0.097378,0
4069,maksssksksss99.png,400.0,267.0,181.0,273.0,54.0,162.0,mask_weared_incorrect,0.567500,0.404494,0.230000,0.404494,2
4070,maksssksksss99.png,400.0,267.0,99.0,176.0,87.0,165.0,with_mask,0.343750,0.471910,0.192500,0.292135,0


In [13]:
df.columns

Index(['filename', 'width', 'height', 'xmin', 'xmax', 'ymin', 'ymax', 'name',
       'center_x', 'center_y', 'W', 'H', 'label'],
      dtype='object')

In [14]:
cols = ['filename','center_x', 'center_y', 'W', 'H', 'label']
df = df[cols]

In [15]:
df.head()

,filename,center_x,center_y,W,H,label
0,maksssksksss0.png,0.183594,0.337432,0.058594,0.101093,1
1,maksssksksss0.png,0.401367,0.333333,0.080078,0.120219,0
2,maksssksksss0.png,0.668945,0.315574,0.068359,0.139344,1
3,maksssksksss1.png,0.843750,0.330128,0.082500,0.224359,0
4,maksssksksss1.png,0.606250,0.355769,0.092500,0.224359,0


In [16]:
uniqueFilename = df['filename'].unique()

In [17]:
uniqueFilename

array(['maksssksksss0.png', 'maksssksksss1.png', 'maksssksksss10.png',
       'maksssksksss100.png', 'maksssksksss101.png',
       'maksssksksss102.png', 'maksssksksss103.png',
       'maksssksksss104.png', 'maksssksksss105.png',
       'maksssksksss106.png', 'maksssksksss107.png',
       'maksssksksss108.png', 'maksssksksss109.png', 'maksssksksss11.png',
       'maksssksksss110.png', 'maksssksksss111.png',
       'maksssksksss112.png', 'maksssksksss113.png',
       'maksssksksss114.png', 'maksssksksss115.png',
       'maksssksksss116.png', 'maksssksksss117.png',
       'maksssksksss118.png', 'maksssksksss119.png', 'maksssksksss12.png',
       'maksssksksss120.png', 'maksssksksss121.png',
       'maksssksksss122.png', 'maksssksksss123.png',
       'maksssksksss124.png', 'maksssksksss125.png',
       'maksssksksss126.png', 'maksssksksss127.png',
       'maksssksksss128.png', 'maksssksksss129.png', 'maksssksksss13.png',
       'maksssksksss130.png', 'maksssksksss131.png',
       'maksssk

In [23]:
fileDf = pd.DataFrame(uniqueFilename,columns=['filename'])
train_file = tuple(fileDf.sample(frac=0.8,random_state=42)['filename'])
test_file  = tuple(fileDf.query(f'filename not in {train_file}')['filename'])

In [25]:
test_file

('maksssksksss1.png',
 'maksssksksss101.png',
 'maksssksksss11.png',
 'maksssksksss110.png',
 'maksssksksss116.png',
 'maksssksksss117.png',
 'maksssksksss122.png',
 'maksssksksss127.png',
 'maksssksksss129.png',
 'maksssksksss13.png',
 'maksssksksss134.png',
 'maksssksksss140.png',
 'maksssksksss145.png',
 'maksssksksss150.png',
 'maksssksksss154.png',
 'maksssksksss156.png',
 'maksssksksss162.png',
 'maksssksksss170.png',
 'maksssksksss175.png',
 'maksssksksss177.png',
 'maksssksksss180.png',
 'maksssksksss184.png',
 'maksssksksss187.png',
 'maksssksksss188.png',
 'maksssksksss190.png',
 'maksssksksss193.png',
 'maksssksksss194.png',
 'maksssksksss207.png',
 'maksssksksss213.png',
 'maksssksksss215.png',
 'maksssksksss219.png',
 'maksssksksss222.png',
 'maksssksksss239.png',
 'maksssksksss241.png',
 'maksssksksss242.png',
 'maksssksksss243.png',
 'maksssksksss248.png',
 'maksssksksss251.png',
 'maksssksksss267.png',
 'maksssksksss269.png',
 'maksssksksss270.png',
 'maksssksksss279.pn

In [26]:
train_df = df.query(f"filename in {train_file}")
test_df = df.query(f"filename in {test_file}")

In [27]:
train_df

,filename,center_x,center_y,W,H,label
0,maksssksksss0.png,0.183594,0.337432,0.058594,0.101093,1
1,maksssksksss0.png,0.401367,0.333333,0.080078,0.120219,0
2,maksssksksss0.png,0.668945,0.315574,0.068359,0.139344,1
12,maksssksksss10.png,0.485050,0.812500,0.318937,0.290000,0
13,maksssksksss100.png,0.542500,0.261062,0.140000,0.256637,0
...,...,...,...,...,...,...
4067,maksssksksss98.png,0.687500,0.275281,0.060000,0.086142,0
4068,maksssksksss98.png,0.901250,0.348315,0.082500,0.097378,0
4069,maksssksksss99.png,0.567500,0.404494,0.230000,0.404494,2
4070,maksssksksss99.png,0.343750,0.471910,0.192500,0.292135,0


In [28]:
test_df

,filename,center_x,center_y,W,H,label
3,maksssksksss1.png,0.843750,0.330128,0.082500,0.224359,0
4,maksssksksss1.png,0.606250,0.355769,0.092500,0.224359,0
5,maksssksksss1.png,0.767500,0.445513,0.040000,0.147436,0
6,maksssksksss1.png,0.396250,0.605769,0.077500,0.262821,0
7,maksssksksss1.png,0.211250,0.538462,0.052500,0.192308,0
...,...,...,...,...,...,...
4049,maksssksksss93.png,0.643750,0.320225,0.047500,0.071161,0
4050,maksssksksss93.png,0.710000,0.337079,0.065000,0.097378,0
4051,maksssksksss93.png,0.751250,0.269663,0.042500,0.074906,0
4052,maksssksksss93.png,0.986250,0.307116,0.032500,0.074906,0


In [33]:
train_groupby = train_df.groupby('filename')
test_groupby = test_df.groupby('filename')

In [43]:
test_groupby.get_group('maksssksksss1.png')

,filename,center_x,center_y,W,H,label
3,maksssksksss1.png,0.84375,0.330128,0.0825,0.224359,0
4,maksssksksss1.png,0.60625,0.355769,0.0925,0.224359,0
5,maksssksksss1.png,0.76750,0.445513,0.0400,0.147436,0
6,maksssksksss1.png,0.39625,0.605769,0.0775,0.262821,0
7,maksssksksss1.png,0.21125,0.538462,0.0525,0.192308,0
8,maksssksksss1.png,0.51500,0.512821,0.0750,0.166667,0
9,maksssksksss1.png,0.08125,0.532051,0.0575,0.128205,0
10,maksssksksss1.png,0.95875,0.541667,0.0725,0.185897,0
11,maksssksksss1.png,0.24250,0.464744,0.0700,0.211538,1


In [50]:
train_folder = "artifacts/data_ingestion/YoloDataset/train_set"
os.makedirs(train_folder,exist_ok=True)
test_folder = "artifacts/data_ingestion/YoloDataset/test_set"
os.makedirs(test_folder,exist_ok=True)

In [45]:
# save data 
def save_data(filename,folder,groupBy):
    
    image_path  = os.path.join('artifacts/data_ingestion/Face_Musk_Detection/Datatset/images',filename)
    
    dist_image_path = os.path.join(folder,filename)
    copy(image_path,dist_image_path)
    
    text_path = os.path.join(folder,
                             os.path.splitext(filename)[0] + '.txt')
    groupBy.get_group(filename).set_index('filename').to_csv(text_path,header=False,index=False,sep=' ')

In [51]:
train_series = pd.Series(train_groupby.groups.keys())
train_series.apply(save_data,args=(train_folder,train_groupby))

0      None
1      None
2      None
3      None
4      None
       ... 
677    None
678    None
679    None
680    None
681    None
Length: 682, dtype: object

In [52]:
test_series = pd.Series(test_groupby.groups.keys())
test_series.apply(save_data,args=(test_folder,test_groupby))

0      None
1      None
2      None
3      None
4      None
       ... 
166    None
167    None
168    None
169    None
170    None
Length: 171, dtype: object